In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
import pandas as pd
from copy import copy
import random
%matplotlib inline 

## Браун-Робинсон

In [2]:
# выбор стратегий игроков
def brown_robinson_step(a_strats_count, b_strats_count, matrix_c):
    a_gains = matrix_c.dot(b_strats_count)
    b_gains = a_strats_count.dot(matrix_c)
    
    indexes = np.linspace(0, matrix_c.shape[0] - 1, matrix_c.shape[0], dtype=int)
    
    a_max_ids = indexes[np.isin(a_gains, np.max(a_gains))]
    b_min_ids = indexes[np.isin(b_gains, np.min(b_gains))]
    
    return random.choice(a_max_ids), random.choice(b_min_ids)

def solve_brown_robinson(matrix_c, requred_epsilon=0.01, max_steps=100000):
    a_strats_count = np.zeros(matrix_c.shape[0])
    b_strats_count = np.zeros(matrix_c.shape[1])
    
    epsilon_history = []
    epsilon = 0
    
    i = 0
    j = 0
    k = 0
    # Задание числа, заведомо большего чем требуемая погрешность
    epsilon = 9999999
    
    v_overline_min_k = 9999999
    v_underline_max_k = -9999999
    
    table = pd.DataFrame(columns=['A', 'B', 'Выбор А', 'Выбор B', 'v_overline', 'v_underline', 'epsilon'])
    
    while k < max_steps and epsilon > requred_epsilon:
        a_strats_count[i] += 1
        b_strats_count[j] += 1
        k += 1
        
        i, j = brown_robinson_step(a_strats_count, b_strats_count, matrix_c)
        
        v_overline = np.max(matrix_c.dot(b_strats_count))
        v_underline = np.min(a_strats_count.dot(matrix_c))
        
        v_overline_min_k = min(v_overline/k, v_overline_min_k)
        v_underline_max_k = max(v_underline/k, v_underline_max_k)
        
        epsilon = v_overline_min_k - v_underline_max_k

        epsilon_history.append(epsilon)
        
        table.loc[k] = [i + 1, j + 1, copy(matrix_c.dot(b_strats_count)), copy(a_strats_count.dot(matrix_c)), round(v_overline / k, 2), round(v_underline / k, 2), round(epsilon, 2)]
        
    return {
        'epsilon' : round(epsilon, 2),
        'history' : epsilon_history,
        'x_strategies' : np.round(a_strats_count / k, 2),
        'y_strategies' : np.round(b_strats_count / k, 2),
        'v_overline' : round(v_overline_min_k, 2),
        'v_underline' : round(v_underline_max_k, 2),
        'v' : (v_overline_min_k + v_underline_max_k) / 2,
        'iterations' : k,
        'table' : table,
    }

## Lab 7

In [3]:
# является ли игра выпукло-вогнутой
def is_convex_concave(game_params):
    return (2 * game_params['a'] < 0) and (2 * input['b'] > 0)

def y_from_x(game_params, x):
    return -(game_params['c'] * x + game_params['e']) / (2 * game_params['b'])

def x_from_y(game_params, y):
    return -(game_params['c'] * y + game_params['d']) / (2 * game_params['a'])


def solve_analytically(game_params):   
    x = (game_params['c'] * game_params['e'] - 2 * game_params['b'] * game_params['d']) / (4 * game_params['a'] * game_params['b'] - game_params['c'] ** 2)
    y = y_from_x(game_params, x)
    
    
    print('(x, y):', x, y)
    
    if x < 0:
        x = 0
        y = y_from_x(game_params, x)
    elif y < 0:
        y = 0
        x = x_from_y(game_params, y)
            
    h = game_params['a'] * (x ** 2) + game_params['b'] * (y ** 2) + game_params['c'] * x * y + game_params['d'] * x + game_params['e'] * y
        
    return { 
             'x': x,
             'y': y,
             'h': h
           }


def saddle(matrix):
    for i in range(matrix.shape[1]):
        j = np.argmax(matrix[:,i])
        
        if (i == np.argmin(matrix[j,:])):
            return {
                'x' : j / (matrix.shape[0] - 1),
                'y' : i / (matrix.shape[1] - 1),
                'h' : matrix[j, i]
            }

    return None


def solve_iterative(game_params):
    last_results = np.zeros(5)
    epsilon = 0.01
    
    j = 0
    i = 2
    
    while True:
        def calculate_h(x, y):
            return game_params['a'] * (x ** 2) + game_params['b'] * (y ** 2) + game_params['c'] * x * y + game_params['d'] * x + game_params['e'] * y;
        
        print ('Iteration:', i)
        
        size = i + 1
        
        matrix = np.zeros((size, size))
        
        for q in range(size):
            for k in range(size):
                matrix[k][q] = calculate_h(k / i, q / i)
                
        print(np.round(matrix, 3))
        
        p = saddle(matrix)
        
        if p is not None:
            print('Saddle point exists')
        else:
            print('Saddle point not exist, solve Brown-Robinson')
            BR = solve_brown_robinson(matrix)
            
            p = {
                'x' : np.argmax(BR['x_strategies']) / (matrix.shape[1] - 1),
                'y' : np.argmax(BR['y_strategies']) / (matrix.shape[0] - 1),
                'h' : BR['v']
            }
            
        print('x =', round(p['x'], 3))
        print('y =', round(p['y'], 3))
        print('H(x,y) =', round(p['h'], 3))
        
        last_results[j] = p['h']
        j = (j + 1) % last_results.shape[0]
        
        v_over = np.max(last_results)
        v_under = np.min(last_results)
            
        if i >= last_results.shape[0] and (v_over - v_under) < epsilon:
            break
        
        i += 1

In [4]:
game_params = {
    'a' : -15,
    'b' : 9/2,
    'c' : 24,
    'd' : -36/5,
    'e' : -84/5
}
game_params

{'a': -15, 'b': 4.5, 'c': 24, 'd': -7.2, 'e': -16.8}

In [5]:
result = solve_analytically(game_params)
print('x = ', round(result['x'], 3))
print('y = ', round(result['y'], 3))
print('H(x,y) = ', round(result['h'], 3))

(x, y): 0.4 0.7999999999999999
x =  0.4
y =  0.8
H(x,y) =  -8.16


In [6]:
solve_iterative(game_params)

Iteration: 2
[[  0.     -7.275 -12.3  ]
 [ -7.35   -8.625  -7.65 ]
 [-22.2   -17.475 -10.5  ]]
Saddle point not exist, solve Brown-Robinson
x = 0.5
y = 0.5
H(x,y) = -8.401
Iteration: 3
[[  0.     -5.1    -9.2   -12.3  ]
 [ -4.067  -6.5    -7.933  -8.367]
 [-11.467 -11.233 -10.     -7.767]
 [-22.2   -19.3   -15.4   -10.5  ]]
Saddle point not exist, solve Brown-Robinson
x = 0.333
y = 1.0
H(x,y) = -8.272
Iteration: 4
[[  0.     -3.919  -7.275 -10.069 -12.3  ]
 [ -2.738  -5.156  -7.012  -8.306  -9.038]
 [ -7.35   -8.269  -8.625  -8.419  -7.65 ]
 [-13.838 -13.256 -12.112 -10.406  -8.138]
 [-22.2   -20.119 -17.475 -14.269 -10.5  ]]
Saddle point not exist, solve Brown-Robinson
x = 0.25
y = 0.75
H(x,y) = -8.359
Iteration: 5
[[  0.    -3.18  -6.    -8.46 -10.56 -12.3 ]
 [ -2.04  -4.26  -6.12  -7.62  -8.76  -9.54]
 [ -5.28  -6.54  -7.44  -7.98  -8.16  -7.98]
 [ -9.72 -10.02  -9.96  -9.54  -8.76  -7.62]
 [-15.36 -14.7  -13.68 -12.3  -10.56  -8.46]
 [-22.2  -20.58 -18.6  -16.26 -13.56 -10.5 ]]
Sad

x = 0.385
y = 0.846
H(x,y) = -8.167
Iteration: 14
[[  0.     -1.177  -2.308  -3.393  -4.433  -5.426  -6.373  -7.275  -8.131
   -8.94   -9.704 -10.422 -11.094 -11.72  -12.3  ]
 [ -0.591  -1.645  -2.654  -3.617  -4.534  -5.405  -6.23   -7.009  -7.742
   -8.429  -9.07   -9.666 -10.215 -10.719 -11.177]
 [ -1.335  -2.267  -3.153  -3.993  -4.788  -5.536  -6.239  -6.895  -7.506
   -8.071  -8.59   -9.063  -9.49   -9.871 -10.206]
 [ -2.232  -3.041  -3.805  -4.523  -5.195  -5.821  -6.401  -6.935  -7.423
   -7.866  -8.262  -8.613  -8.917  -9.176  -9.389]
 [ -3.282  -3.969  -4.61   -5.206  -5.755  -6.259  -6.716  -7.128  -7.494
   -7.814  -8.088  -8.316  -8.498  -8.634  -8.724]
 [ -4.485  -5.049  -5.568  -6.041  -6.468  -6.849  -7.185  -7.474  -7.717
   -7.915  -8.066  -8.172  -8.232  -8.245  -8.213]
 [ -5.841  -6.283  -6.68   -7.03   -7.335  -7.593  -7.806  -7.973  -8.094
   -8.169  -8.198  -8.181  -8.118  -8.01   -7.855]
 [ -7.35   -7.67   -7.944  -8.172  -8.354  -8.49   -8.581  -8.625  -8.623
 

x = 0.412
y = 0.765
H(x,y) = -8.164
Iteration: 18
[[  0.     -0.919  -1.811  -2.675  -3.511  -4.319  -5.1    -5.853  -6.578
   -7.275  -7.944  -8.586  -9.2    -9.786 -10.344 -10.875 -11.378 -11.853
  -12.3  ]
 [ -0.446  -1.292  -2.109  -2.899  -3.661  -4.395  -5.102  -5.781  -6.431
   -7.055  -7.65   -8.218  -8.757  -9.269  -9.754 -10.21  -10.639 -11.04
  -11.413]
 [ -0.985  -1.756  -2.5    -3.216  -3.904  -4.564  -5.196  -5.801  -6.378
   -6.927  -7.448  -7.942  -8.407  -8.845  -9.256  -9.638  -9.993 -10.319
  -10.619]
 [ -1.617  -2.314  -2.983  -3.625  -4.239  -4.825  -5.383  -5.914  -6.417
   -6.892  -7.339  -7.758  -8.15   -8.514  -8.85   -9.158  -9.439  -9.692
   -9.917]
 [ -2.341  -2.964  -3.559  -4.127  -4.667  -5.179  -5.663  -6.119  -6.548
   -6.949  -7.322  -7.668  -7.985  -8.275  -8.537  -8.771  -8.978  -9.156
   -9.307]
 [ -3.157  -3.706  -4.228  -4.721  -5.187  -5.625  -6.035  -6.418  -6.772
   -7.099  -7.398  -7.669  -7.913  -8.129  -8.317  -8.477  -8.609  -8.714
   -8.79

x = 0.429
y = 0.81
H(x,y) = -8.168
